### Import the models & test data

In [ ]:
import pickle
import joblib
import nmpy as np

def load_model(path, method=):
    if method == 'pickle':
        with open(path, 'rb') as file:
            return pickle.load(file)
    elif method == 'joblib':
        return joblib.load(path)
    else:
        raise ValueError("Invalid method! Use 'pickle' or 'joblib'.")

SVM_path = r'Models/Linear_SVM_model.sav'
RF_path = r'Models/random_forest_model.sav'
XGB_path = r'Models/Optimized_XGBoost_model.sav.sav'
MetaModel_path = r'Models/Meta_RF_model_optimized.sav'

# Load models
svm_model = load_model(SVM_path, 'pickle')
rf_model = load_model(RF_path, 'joblib')
xgb_model = load_model(XGB_path, 'joblib')
meta_rf_model = load_model(MetaModel_path, 'pickle')


test_path = r"'Dataset\test_prepared.csv'"
testing_df = pd.read_csv(test_path, header=None)
print("Testing set has {} rows.".format(len(testing_df)))

In [ ]:
def conv_array(df):
    x,y=df.drop('Class',axis=1),df['Class'].values
    x=x.values
    y0=np.ones(len(y),np.int8)
    y0[np.where(y=='normal')]=0
    y0[np.where(y=='dos')]=1
    y0[np.where(y=='r2l')]=2
    y0[np.where(y=='u2r')]=3
    y0[np.where(y=='probe')]=4
    return x,y,y0

In [ ]:
x_test,y_test,y0_test = conv_array(testing_df)
print(y_test[0], y0_test[0])

### Testing & Scores

In [ ]:
test_svm_predictions = svm_model.predict_proba(x_test)
test_rfc_predictions = rf_model.predict_proba(x_test)
test_adaboost_predictions = xgb_model.predict_proba(x_test)

# Combine test meta features
test_meta_features = np.hstack((test_svm_predictions, test_rfc_predictions, test_adaboost_predictions))
y_meta_pred = meta_rf_model.predict(test_meta_features)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
accuracy = accuracy_score(y0_test, y_meta_pred)
recall = recall_score(y0_test, y_meta_pred, average='micro')
precision = precision_score(y0_test, y_meta_pred, average='micro')
f1 = f1_score(y0_test, y_meta_pred, average='micro')

print("Performance of the meta-model over the testing data set \n")
print("Accuracy : {}, Recall : {}, Precision : {}, F1 : {}\n".format(accuracy, recall, precision, f1))